In [125]:
import googlemaps
import pandas as pd
import json
from utils import *
from api_key import api_key
import polyline

# Replace 'YOUR_API_KEY' with your actual API key


In [126]:
# Replace "api_key" with your actual API key
'''to get api key from google:Sign up for a Google Cloud account and enable the Maps API:
 Go to the Google Cloud Console (https://console.cloud.google.com) and create a new project.
Enable the "Maps JavaScript API" and the "Directions API" for your project.
'''
#the billing is free for the first 300 dolars so do not worry about the pricing but still check the website billing
#I think for this part billing can be closed anyway so no payment guaranteed

api_key = api_key
gmaps = googlemaps.Client(key=api_key)

In [127]:

def data_collection_manual(searched_item):
    total = []
    df = gmaps.places(query = searched_item)
    df = json.dumps(df)
    df = json.loads(df)
    df = pd.DataFrame(df["results"])
    total.append(df.iloc[:1])
    df = pd.concat(total)
    return df

def get_lat_long(df):

    df["Latitude"] = df["geometry"].astype(str).str.extract(':(.{,45})')
    df["Latitude"] = df["Latitude"].str.split(',').str[0]
    df["Latitude"] = df["Latitude"].str[8:]
  
    df["Longitude"] = df["geometry"].astype(str).str.split(",").str[1]
    df["Longitude"] = df["Longitude"].astype(str).str[7:]
    df["Longitude"] = df["Longitude"].astype(str).str[:-1]
    return df["Latitude"][0],df["Longitude"][0]



def get_road_distance(location1, location2):
    location1_long_lat = get_lat_long(data_collection_manual(location1))
    location2_long_lat  = get_lat_long(data_collection_manual(location2))
    result = gmaps.distance_matrix(location1_long_lat, location2_long_lat, mode='driving')
    distance = result['rows'][0]['elements'][0]['distance']['text']
    return distance   

In [128]:
data_collection_manual("ankamall avm")

,business_status,formatted_address,geometry,icon,icon_background_color,icon_mask_base_uri,name,opening_hours,photos,place_id,plus_code,rating,reference,types,user_ratings_total,price_level
0,OPERATIONAL,"Gazi, Konya Devlet Yolu, Mevlana Blv. No:2, 06...","{'location': {'lat': 39.9504171, 'lng': 32.831...",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,ANKAmall,{'open_now': True},"[{'height': 2240, 'html_attributions': ['<a hr...",ChIJ3euM5ZVO0xQRilR8Be2HpLA,"{'compound_code': 'XR2J+5G Yenimahalle, Ankara...",4.3,ChIJ3euM5ZVO0xQRilR8Be2HpLA,"[shopping_mall, point_of_interest, establishment]",62000,NaN


In [129]:
float(get_lat_long(data_collection_manual("ankamall avm"))[0])

39.9504171

In [130]:
get_road_distance("ankamall avm","metromall avm")

'22.9 km'

In [131]:
get_road_distance("çankaya belediyesi","etimesgut belediyesi")

'20.3 km'

In [133]:
import folium

def visualize_route(origin_lat, origin_lng, destination_lat, destination_lng):
    
    directions = gmaps.directions(
        (origin_lat, origin_lng),
        (destination_lat, destination_lng),
        mode='driving'
    )

    # Extract the polyline from the directions response
    polyline_points = directions[0]['overview_polyline']['points']

    # Decode the polyline (ı had an error before decoding so ı had to decode it 
    decoded_polyline = polyline.decode(polyline_points)

    #initial map 
    my_map = folium.Map(location=[origin_lat, origin_lng], zoom_start=10)

    # Markers for locations 
    folium.Marker([origin_lat, origin_lng], tooltip='Origin').add_to(my_map)
    folium.Marker([destination_lat, destination_lng], tooltip='Destination').add_to(my_map)

    # Plotting
    folium.PolyLine(locations=decoded_polyline, color='blue', weight=5).add_to(my_map)

    
    return my_map



In [135]:
visualize_route(float(get_lat_long(data_collection_manual("ankamall avm"))[0]),float(get_lat_long(data_collection_manual("ankamall avm"))[1])
                ,float(get_lat_long(data_collection_manual("metromall avm"))[0]),float(get_lat_long(data_collection_manual("metromall"))[1]))